# Getting started - level 2

In this tutorial we will explore a little bit more advanced example of a program that require some configuration, requirements setup, etc. 

Again we will start with writing code for our program and saving it to [./source_files/gs_level_2.py](./source_files/gs_level_2.py) file.
This time it will be VQE example from [Qiskit documentation](https://qiskit.org/documentation/nature/tutorials/07_leveraging_qiskit_runtime.html) and we also introduce dependency management and arguments to our programs.

```python
# source_files/gs_level_2.py

from quantum_serverless import get_arguments

from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import QubitConverter
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.properties import ParticleNumber
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms.ground_state_solvers import GroundStateEigensolver
from qiskit.circuit.library import EfficientSU2
import numpy as np
from qiskit.utils import algorithm_globals
from qiskit.algorithms.optimizers import SPSA
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator


def run(bond_distance: float = 2.5):
    driver = PySCFDriver(
        atom=f"Li 0 0 0; H 0 0 {bond_distance}",
        basis="sto3g",
        charge=0,
        spin=0,
        unit=DistanceUnit.ANGSTROM,
    )
    problem = driver.run()

    active_space_trafo = ActiveSpaceTransformer(
        num_electrons=problem.num_particles, num_spatial_orbitals=3
    )
    problem = active_space_trafo.transform(problem)
    qubit_converter = QubitConverter(ParityMapper(), two_qubit_reduction=True)

    ansatz = EfficientSU2(num_qubits=4, reps=1, entanglement="linear", insert_barriers=True)

    np.random.seed(5)
    algorithm_globals.random_seed = 5


    optimizer = SPSA(maxiter=100)
    initial_point = np.random.random(ansatz.num_parameters)

    estimator = Estimator()
    local_vqe = VQE(
        estimator,
        ansatz,
        optimizer,
        initial_point=initial_point,
    )

    local_vqe_groundstate_solver = GroundStateEigensolver(qubit_converter, local_vqe)
    local_vqe_result = local_vqe_groundstate_solver.solve(problem)

    print(local_vqe_result)


arguments = get_arguments()
bond_length = arguments.get("bond_length", 2.55)
print(f"Running for bond length {bond_length}.")
run(bond_length)

```

As you can see here we used couple of additional things compared to `getting started level 1`. 

First, we are introducing dependency management by using the `qiskit-nature` module and `pyscf` extension.
We also using `get_arguments` function to parse arguments to our program, which return dictionary of arguments. In this case argument is `bond_length`. This means that we can, re-run our program over different bond lengths and produce a dissociation curve.


Next we need to run this program. For that we need to import necessary modules and configure `QuantumServerless` client. We are doing so by providing name and host for deployed infrastructure.

In [1]:
from quantum_serverless import QuantumServerless, GatewayProvider

In [2]:
provider = GatewayProvider(
    username="user", # this username has already been defined in local docker setup and does not need to be changed
    password="password123", # this password has already been defined in local docker setup and does not need to be changed
    host="http://gateway:8000", # address of provider
)

serverless = QuantumServerless(provider)
serverless

<QuantumServerless | providers [gateway-provider]>

In addition to that we will provide additional `dependencies` to our `Program` construction and `arguments` to `run_program` method.
- `dependencies` parameter will install provided libraries to run our script. Dependencies can be python libraries available on PyPi or any package source installable via pip package manager .
- `arguments` parameter is a dictionary with arguments that will be passed for script execution

In [3]:
from quantum_serverless import Program

program = Program(
    title="Getting started program level 2",
    entrypoint="gs_level_2.py",
    working_dir="./source_files",
    dependencies=["qiskit-nature", "qiskit-nature[pyscf]"]
)

job = serverless.run_program(
    program=program, 
    arguments={
        "bond_length": 2.55
    }
)
job

<Job | 534debe6-3a1b-4aea-933f-cdc2d67863d8>

In [7]:
job.status()

'SUCCEEDED'

In [8]:
print(job.logs())

Running for bond length 2.55.
/tmp/ray/session_2023-04-27_10-58-53_142605_1/runtime_resources/working_dir_files/_ray_pkg_0a274edc02afd24a/gs_level_2.py:33: DeprecationWarning: The QubitConverter class is deprecated as of version 0.6.0 and will be removed no sooner than 3 months after the release . Instead you should directly use the QubitMapper instance which you used to pass into the QubitConverter as the first argument. Refer to the documentation of the qiskit_nature.second_q.mappers module for more information.
  qubit_converter = QubitConverter(ParityMapper(), two_qubit_reduction=True)
/home/ray/anaconda3/lib/python3.9/site-packages/qiskit_nature/deprecation.py:297: PauliSumOpDeprecationWarning: PauliSumOp is deprecated as of version 0.6.0 and support for them will be removed no sooner than 3 months after the release. Instead, use SparsePauliOp. You can switch to SparsePauliOp immediately, by setting `qiskit_nature.settings.use_pauli_sum_op` to `False`.
  return func(*args, **kwarg

---
If you want to run this program with different bond length you can run it 3 times. Programs are asynchronous, therefore each of instance of program will be running in parallel.

In [13]:
jobs = []

for bond_length in [2.55, 3.0, 3.55]:
    program = Program(
        title=f"Groundstate with bond length {bond_length}",
        entrypoint="gs_level_2.py",
        working_dir="./source_files",
        dependencies=["qiskit-nature", "qiskit-nature[pyscf]"]
    )
    jobs.append(serverless.run_program(program, { "bond_length": bond_length }))

jobs

[<Job | ac223c88-9b84-472c-b77b-dd3c56cd0244>,
 <Job | d206f2ea-7d6b-43fe-a294-4ddc953eeb11>,
 <Job | 6c6999bb-d942-491e-8411-92f5f62f567d>]

In [19]:
for job in jobs:
    print(job.status())

SUCCEEDED
SUCCEEDED
SUCCEEDED


In [20]:
for job in jobs:
    print(job.logs())

Running for bond length 2.55.
/tmp/ray/session_2023-04-27_10-58-53_142605_1/runtime_resources/working_dir_files/_ray_pkg_0a274edc02afd24a/gs_level_2.py:33: DeprecationWarning: The QubitConverter class is deprecated as of version 0.6.0 and will be removed no sooner than 3 months after the release . Instead you should directly use the QubitMapper instance which you used to pass into the QubitConverter as the first argument. Refer to the documentation of the qiskit_nature.second_q.mappers module for more information.
  qubit_converter = QubitConverter(ParityMapper(), two_qubit_reduction=True)
/home/ray/anaconda3/lib/python3.9/site-packages/qiskit_nature/deprecation.py:297: PauliSumOpDeprecationWarning: PauliSumOp is deprecated as of version 0.6.0 and support for them will be removed no sooner than 3 months after the release. Instead, use SparsePauliOp. You can switch to SparsePauliOp immediately, by setting `qiskit_nature.settings.use_pauli_sum_op` to `False`.
  return func(*args, **kwarg

---
Other way would be refactoring program file itself to accept list of bond length and run them in a loop inside a program.
If you want 3 independent results, then running 3 programs would be a better fit. But if you want to do some postprocessing after execution of multiple function, then refactoring program file to run 3 function and postprocess them would be better choice. But at the end it all boils down to user preference.